In [ ]:
!pip install pystan~=2.14
!pip install fbprophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: pystan
    Found existing installation: pystan 3.3.0
    Uninstalling pystan-3.3.0:
      Successfully uninstalled pystan-3.3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fbprophet: filename=fbprophet-0.7.1-py3-none-any.whl size=6751506 sha256=8b3ef19bae1e279a85c0c1c078d227e1ae0cb8e379992ed389d9790471f78e83
  Stored in directory: /root/.cache/pip/wheels/d0/d2/ae/c579b7fd160999d35908f3cb8ebcad7ef64ecaca7b78e4c3c8
Successfully built fbprophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 1.0.8
    Uninstalling cmdstanpy-1.0.8:
      Successfully uninstalled cmdsta

In [6]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tqdm as tqdm

import matplotlib.pyplot as plt

%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn


# special matplotlib argument for improved plots
from matplotlib import rcParams

In [7]:
data = pd.read_csv("SSI Historical Data.csv")
data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2017/12/1,13928.5,13625.2,14105.5,13574.6,10420000,0.02
1,2017/12/4,14712.2,14054.9,14813.3,13979.1,11520000,0.06
2,2017/12/5,14257.2,14661.6,14712.2,14156.1,11100000,-0.03
3,2017/12/6,14130.8,14206.6,14257.2,13625.2,14660000,-0.01
4,2017/12/7,13701.0,14130.8,14156.1,13599.9,8850000,-0.03


In [8]:
dataset = data.loc[:,["Date","Price"]]
dataset.head()

,Date,Price
0,2017/12/1,13928.5
1,2017/12/4,14712.2
2,2017/12/5,14257.2
3,2017/12/6,14130.8
4,2017/12/7,13701.0


In [ ]:
dataset.info()

In [9]:
dataset.Date = pd.to_datetime(dataset.Date,format = "%Y/%m/%d")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1249 non-null   datetime64[ns]
 1   Price   1249 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.6 KB


In [10]:
dataset = dataset.rename(columns = {"Price" : "y"})
dataset = dataset.rename(columns = {"Date" : "ds"})
dataset.head()

,ds,y
0,2017-12-01,13928.5
1,2017-12-04,14712.2
2,2017-12-05,14257.2
3,2017-12-06,14130.8
4,2017-12-07,13701.0


In [11]:
test_days = 375
training_set = dataset.iloc[:-test_days, :]
test_set = dataset.iloc[-test_days:, :]
test_set.tail(5)

,ds,y
1244,2022-11-25,17100.0
1245,2022-11-28,18250.0
1246,2022-11-29,18950.0
1247,2022-11-30,19250.0
1248,2022-12-01,18200.0


In [ ]:
print(len(training_set))
print(len(test_set))

In [ ]:
training_set.tail(5)

In [ ]:
m = Prophet(growth = "linear",
            yearly_seasonality = True,
            weekly_seasonality = True,
            daily_seasonality = False,
            seasonality_mode = "multiplicative",
            seasonality_prior_scale = 10,
            holidays_prior_scale = 10,
            changepoint_prior_scale = 0.05)
m.fit(training_set)

In [ ]:
future = m.make_future_dataframe(periods = len(test_set), freq = "D")
future.tail()

In [ ]:
future = pd.concat([future, dataset.iloc[:,1:]],
                   axis = 1)
future.tail(5)

In [ ]:
forecast = m.predict(future)
forecast.head()

In [ ]:
predictions_prophet = forecast.yhat[-test_days:].rename("prophet")
predictions_prophet[:5]

In [ ]:
from fbprophet.diagnostics import cross_validation
cs = cross_validation(m,
                         #Dự đoán 30 ngày tiếp theo
                         horizon = '30 days',
                         #Lấy 540 ngày dể đào tạo
                         initial = '365 days',
                         #Xử lý song song
                         parallel = "processes")

In [ ]:
y_true = cs['y'].values
y_pre = cs['yhat'].values

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
mae = mean_absolute_error(y_true,y_pre)
mape = mean_absolute_percentage_error(y_true,y_pre)
mse = mean_squared_error(y_true, y_pre)
rmse = np.sqrt(mse)
r2 = r2_score(y_true,y_pre)

print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape * 100:.2f}%")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2: {r2:.2f}")

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {'seasonality_prior_scale': np.arange(5, 10, 5), #[5, 10]  np.arange(5, 20, 5),
              'changepoint_prior_scale': np.arange(0.01, 0.05, 0.01),  # [0.01, 0.02, 0.03, 0.04, 0.05]
              'holidays_prior_scale': np.arange(5, 10, 5)}#[5, 10]
grid = ParameterGrid(param_grid)
list(grid)[:3]  

In [ ]:
rmse = []
for params in grid:
  #build model
  m = Prophet(growth = "linear",
            yearly_seasonality = True,
            weekly_seasonality = True,
            daily_seasonality = False,
            seasonality_mode = "multiplicative",
            seasonality_prior_scale = params['seasonality_prior_scale'],
            holidays_prior_scale = params['holidays_prior_scale'],
            changepoint_prior_scale = params['changepoint_prior_scale'])
  m.fit(training_set)

  #cross-validation
  cs = cross_validation(m,
                         horizon = '30 days',
                         initial = '540 days',
                         parallel = "processes")
  
  #gather the results
  error = np.sqrt(mean_squared_error(cs['y'], 
                                     cs['yhat']))
  rmse.append(error)

In [ ]:
best_params = grid[np.argmin(rmse)]
print(best_params)

In [ ]:
m = Prophet(growth = "linear",
          yearly_seasonality = True,
          weekly_seasonality = True,
          daily_seasonality = False,
          seasonality_mode = "multiplicative",
          seasonality_prior_scale = best_params['seasonality_prior_scale'],
          holidays_prior_scale = best_params['holidays_prior_scale'],
          changepoint_prior_scale = best_params['changepoint_prior_scale'])
m.fit(training_set)

In [ ]:
future = m.make_future_dataframe(periods = len(test_set),
                                 freq = "D")
future = pd.concat([future, dataset.iloc[:,1:]],
                   axis = 1)
future.tail(1)

In [ ]:
forecast = m.predict(future)
forecast.head()

In [ ]:
print(len(forecast['yhat']))
print(len(forecast.yhat[-test_days:]))
print(len(test_set["ds"]))

In [ ]:
predictions_prophet_tuned = forecast.yhat[-test_days:].rename("prophet_tuned")
predictions_prophet_tuned.index = test_set["ds"]
predictions_prophet_tuned.head()